## Authenticates and initialises Earth Engine

To use ee and geemap, it is required to set up a Google Earth account. Make sure you follow the instructions @ https://geemap.org/installation/#earth-engine-account.

Examples here follow the tutorias from https://github.com/giswqs/earthengine-py-notebooks/tree/master/Tutorials/GlobalSurfaceWater

In [1]:
import ee
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()
    
import geemap

## Add an interactive map based on a specified Region of Interest (ROI)

In [3]:
Map = geemap.Map()
# bottom left, top left, top right, bottom right
roi = ee.Geometry.Polygon(
  [[[38.988041, 8.30037],
    [38.988041, 8.486557],
    [39.169084, 8.486557],
    [39.169084, 8.30037]]], None, False)

Map.centerObject(ee_object=roi, zoom=12)
Map.add_basemap("TERRAIN")
Map

Map(center=[8.393461223691679, 39.07856249999964], controls=(WidgetControl(options=['position'], widget=HBox(c…

## Occurence from global dataset JRC-GSW

Occurence: The frequency with which water was present between 1984-2019

Check the following link https://developers.google.com/earth-engine/datasets/catalog/JRC_GSW1_2_GlobalSurfaceWater

More info can be found @ https://storage.googleapis.com/global-surface-water/downloads_ancillary/DataUsersGuidev2.pdf

In [4]:
GSW_occurrence = ee.Image('JRC/GSW1_2/GlobalSurfaceWater').select('occurrence')
GSW_vis = {'min': 0.0,'max': 100.0, 'palette': ['ffffff', 'ffbbbb', '0000ff']};
Map.addLayer(GSW_occurrence, GSW_vis, 'Occurrence')
legend_keys = ['No water', '1% occurrence', '100% occurrence']
legend_colors = ['#ffffff', '#ffbbbb', '#0000ff']

Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='bottomright')

## Occurrence Change Intensity from JRC-GSW

In [5]:
GSW_Occ_change = ee.Image('JRC/GSW1_2/GlobalSurfaceWater').select('change_norm')
GSW_Occ_change_vis = {'min': -100.0,'max': 100.0, 'palette': ['FF0000 ', '000000', '00FF00 ']};
Map.addLayer(GSW_Occ_change, GSW_Occ_change_vis, 'Occurrence change Intensity')
legend_keys = ['-100% loss of occurrence', 'No change', '100% increase in occurrence']
legend_colors = ['#FF0000', '#000000', '#00FF00']

Map.add_legend(legend_keys=legend_keys, legend_colors=legend_colors, position='topright')


## GLCF: Landsat Global Inland Water

More info can be found @ https://developers.google.com/earth-engine/datasets/catalog/GLCF_GLS_WATER 

In [6]:
GLS_WATER = ee.ImageCollection('GLCF/GLS_WATER')
globalwater2000 = GLS_WATER.select('water').mosaic()
globalwater2000_Vis = {
  'min': 1.0,
  'max': 4.0,
  'palette': ['FAFAFA', '00C5FF', 'DF73FF', '828282', 'CCCCCC'],
}
Map.addLayer(globalwater2000, globalwater2000_Vis, 'GLCF Water')

## HydroSheds - watersheds 

Information can be found here:
- https://developers.google.com/earth-engine/datasets/tags/hydrosheds
- https://developers.google.com/earth-engine/datasets/catalog/WWF_HydroSHEDS_v1_Basins_hybas_8

In [8]:
hydroshed_data = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_7')
styleParams = {'fillColor': '5885E3', 'color': '0000be', 'width': 1.0,}
watersheds = hydroshed_data.style(**styleParams)
Map.addLayer(watersheds, {}, 'Basins Level 7');

In [9]:
hydroshed_data = ee.FeatureCollection('WWF/HydroSHEDS/v1/Basins/hybas_8')
styleParams = {'fillColor': '2E85BB', 'color': '2E5D7E', 'width': 1.0,}
watersheds = hydroshed_data.style(**styleParams)
Map.addLayer(watersheds, {}, 'Basins Level 8');

## HydroSheds - Elevation

In [10]:
hydroshed_dataset = ee.Image('WWF/HydroSHEDS/03VFDEM')
hydroshedelevation = hydroshed_dataset.select('b1')
hydroshedelevationVis = {'min': -50.0,'max': 3000.0,'gamma': 2.0,}
Map.addLayer(hydroshedelevation, hydroshedelevationVis, 'Elevation Void Filled')

## ALOS landforms
Information can be found here:
https://developers.google.com/earth-engine/datasets/catalog/CSP_ERGo_1_0_Global_ALOS_landforms#description

In [11]:
dataset = ee.Image('CSP/ERGo/1_0/Global/ALOS_landforms')
landforms = dataset.select('constant')
landformsVis = {
  'min': 11.0,
  'max': 42.0,
  'palette': [
    '141414', '383838', '808080', 'EBEB8F', 'F7D311', 'AA0000', 'D89382',
    'DDC9C9', 'DCCDCE', '1C6330', '68AA63', 'B5C98E', 'E1F0E5', 'a975ba',
    '6f198c'
  ],
}

Map.addLayer(landforms, landformsVis, 'Landforms')

##  Landcover data - Copernicus 2019
Information can be found @ https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global

In [12]:
Map2 = geemap.Map()
landcover = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('discrete_classification')
# Specific palette set up for each level
Map2.addLayer(landcover, {}, "Land Cover")
Map2.add_legend(builtin_legend='COPERNICUS/Landcover/100m/Proba-V/Global')
Map2.centerObject(ee_object=roi, zoom=12)
Map2

Map(center=[8.393461223691679, 39.07856249999964], controls=(WidgetControl(options=['position'], widget=HBox(c…